In [1270]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment, Border, Side
import numpy as np
import warnings
import os

warnings.filterwarnings('ignore')

In [1271]:
sheets = []

for sheet in pd.read_excel(f'{os.getcwd()}/export_2022.xlsx', sheet_name=None):
    wb = Workbook()
    ws = wb.active
    sheet_name = sheet
    sheets.append(sheet_name)

sheets[:5]

['CW49', 'CW48', 'CW47', 'CW46', 'CW45']

In [1272]:
initial_dfs = {}

for sheet in sheets:
    initial_df = pd.read_excel(f'{os.getcwd()}/export_2022.xlsx', sheet_name=sheet)
    initial_dfs[sheet] = initial_df

In [1273]:
for i in initial_dfs:
    initial_dfs[i] = initial_dfs[i].fillna(0)
    for j in initial_dfs[i].columns:
        if type(j) == pd.datetime:
            initial_dfs[i].rename(columns={j: j.date()}, inplace=True)

In [1274]:
initial_dfs['CW42'] = initial_dfs['CW42'].drop(initial_dfs['CW42'].columns[1], axis=1, inplace=False).copy()
initial_dfs['CW42'] = initial_dfs['CW42'].rename(columns={initial_dfs['CW42'].columns[0]: 'Malzeme'}, inplace=False).copy()

In [1275]:
initial_dfs['CW1'] = initial_dfs['CW1'].rename(columns={initial_dfs['CW1'].columns[0]: 'Malzeme'}, inplace=False).copy()
initial_dfs['CW2'] = initial_dfs['CW2'].rename(columns={initial_dfs['CW2'].columns[0]: 'Malzeme'}, inplace=False)
initial_dfs['CW2'] = initial_dfs['CW2'].rename(columns={initial_dfs['CW2'].columns[1]: 'Toplam Adet'}, inplace=False).copy()

In [1276]:
initial_dfs['CW3'] = initial_dfs['CW3'].rename(columns={initial_dfs['CW3'].columns[0]: 'Malzeme'}, inplace=False).copy()
initial_dfs['CW3'] = initial_dfs['CW3'].rename(columns={initial_dfs['CW3'].columns[1]: 'Toplam Adet'}, inplace=False).copy()

In [1277]:
initial_totals = {}

for i in initial_dfs:
    for j in initial_dfs[i].columns:
        if type(j) == str and j.strip() == 'Toplam Sipariş':
            initial_dfs[i] = initial_dfs[i].rename(columns={j: 'Toplam Adet'}, inplace=False).copy()

In [1278]:
for i in initial_dfs:
    if 'Toplam Adet' in initial_dfs[i].columns:
        initial_totals[i] = initial_dfs[i]['Toplam Adet']
        initial_dfs[i] = initial_dfs[i].drop('Toplam Adet', axis=1, inplace=False).copy()

In [1279]:
for i in initial_dfs:
    if 'Total' not in initial_dfs[i].columns:
        initial_dfs[i]['Total'] = initial_dfs[i].sum(axis=1).copy()
    initial_dfs[i].sort_values(by=['Total'], ascending=False, inplace=True, ignore_index=True)

In [1280]:
for i in initial_dfs:
    initial_dfs[i] = initial_dfs[i].rename(columns={'Malzeme': 'Pipe TTNr'}, inplace=False).copy()

In [1281]:
for i in initial_dfs:
    if initial_dfs[i].index.start != 1:
        initial_dfs[i].index += 1

In [1282]:
for i in initial_dfs:
    initial_dfs[i] = initial_dfs[i][initial_dfs[i]['Total'] != 0].copy()

In [1283]:
all_pipes = []

for i in initial_dfs:
    local_pipes = [str(i).strip() for i in initial_dfs[i]['Pipe TTNr'].unique()]
    all_pipes.extend(local_pipes)
    all_pipes = list(set(all_pipes))

In [1284]:
for i, pipe in enumerate(all_pipes):
    if "-" in pipe:
        all_pipes[i] = "".join(pipe.split("-")).strip()

In [1285]:
unique_pipes = list(set(all_pipes))

In [1286]:
for i in initial_dfs:
    initial_dfs[i]['Pipe TTNr'] = initial_dfs[i]['Pipe TTNr'].astype(str)
    initial_dfs[i]['Pipe TTNr'] = initial_dfs[i]['Pipe TTNr'].apply(lambda x: "".join(x.split("-")).strip())

In [1287]:
with pd.ExcelWriter(f'{os.getcwd()}/modified_export_2022.xlsx', engine='openpyxl', mode='w') as writer:
    for i in initial_dfs:
        initial_dfs[i].to_excel(writer, sheet_name=i, index=True)

In [1288]:
wb = load_workbook(f'{os.getcwd()}/modified_export_2022.xlsx')
for sheet in wb.sheetnames:
    ws = wb[sheet]
    
    for i in range(1, ws.max_row + 1):
        ws.row_dimensions[i].height = 20

    for i in range(2, ws.max_column + 1):
        ws.column_dimensions[ws.cell(row=1, column=i).column_letter].width = 20

    for i in range(1, 2):
        ws.column_dimensions[ws.cell(row=1, column=i).column_letter].width = 12

    for i in range(1, ws.max_row + 1):
        for j in range(1, ws.max_column + 1):
            ws.cell(row=i, column=j).alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

    for i in range(1, ws.max_row + 1):
        for j in range(1, ws.max_column + 1):
            ws.cell(row=i, column=j).font = Font(size=10)

    for i in range(1, ws.max_row + 1):
        for j in range(1, ws.max_column + 1):
            ws.cell(row=i, column=1).font = Font(size=12, bold=True)

    for i in range(1, ws.max_row + 1):
        for j in range(1, ws.max_column + 1):
            ws.cell(row=1, column=j).font = Font(size=12, bold=True)

    for i in range(1, ws.max_row + 1):
        for j in range(1, ws.max_column + 1):
            ws.cell(row=i, column=j).border = Border(left=Side(border_style='thin', color='000000'),
                                                    right=Side(border_style='thin', color='000000'),
                                                    top=Side(border_style='thin', color='000000'),
                                                    bottom=Side(border_style='thin', color='000000'))

    for i in range(1, ws.max_row + 1):
        for j in range(1, ws.max_column + 1):
            if ws.cell(row=i, column=j).value == 0:
                ws.cell(row=i, column=j).value = np.nan
    
wb.save(f'{os.getcwd()}/modified_export_2022.xlsx')

In [1289]:
exports_per_pipe = {}

for i in initial_dfs:
    exports_per_pipe[i] = initial_dfs[i].set_index('Pipe TTNr')['Total'].to_dict()

In [1290]:
export_dfs = {}

for i in exports_per_pipe:
    export_dfs[i] = pd.DataFrame(exports_per_pipe[i].items(), columns=['Pipe TTNr', 'Total'])

In [1291]:
exports_combined = {}

for i in exports_per_pipe:
    for j in exports_per_pipe[i]:
        if j not in exports_combined:
            exports_combined[j] = exports_per_pipe[i][j]
        else:
            # print(f'{j}: {exports_combined[j]} + {exports_per_pipe[i][j]} -> {exports_combined[j] + exports_per_pipe[i][j]}')
            exports_combined[j] += exports_per_pipe[i][j]

In [1292]:
exports_combined_df = pd.DataFrame.from_dict(exports_combined, orient='index', columns=['Total'])
exports_combined_df = exports_combined_df.sort_values(by=['Total'], ascending=False, inplace=False)

In [1293]:
most_common_pipes = exports_combined_df.head(20).index.tolist()
most_common_pipes[:5]

['87167644199',
 '87167638629',
 '8718672348T03',
 '8750724192T03',
 '8718674696T03']

In [1294]:
all_dates = []

for i in initial_dfs:
    all_dates.extend(initial_dfs[i].columns.tolist()[1:-1])

In [1295]:
exports_combined_df = exports_combined_df.reset_index(inplace=False).copy()

if exports_combined_df.index[0] != 1:
    exports_combined_df.index += 1

exports_combined_df.columns = ['Pipe TTNr', 'Total']

In [1296]:
with pd.ExcelWriter(f'{os.getcwd()}/modified_export_2022.xlsx', engine='openpyxl', mode='a') as writer:

    if 'Combined' not in wb.sheetnames:
        wb.create_sheet('Combined', 0)
        wb.move_sheet('Combined', offset=-5)
        exports_combined_df.to_excel(writer, sheet_name='Combined', index=True)
    else:
        exports_combined_df.to_excel(writer, sheet_name='Combined', index=True)

wb = load_workbook(f'{os.getcwd()}/modified_export_2022.xlsx')
ws = wb['Combined']

for i in range(1, ws.max_row + 1):
    ws.row_dimensions[i].height = 20

for i in range(2, ws.max_column + 1):
    ws.column_dimensions[ws.cell(row=1, column=i).column_letter].width = 20

for i in range(1, 2):
    ws.column_dimensions[ws.cell(row=1, column=i).column_letter].width = 12

for i in range(1, ws.max_row + 1):
    for j in range(1, ws.max_column + 1):
        ws.cell(row=i, column=j).alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

for i in range(1, ws.max_row + 1):
    for j in range(1, ws.max_column + 1):
        ws.cell(row=i, column=j).font = Font(size=10)

for i in range(1, ws.max_row + 1):
    for j in range(1, ws.max_column + 1):
        ws.cell(row=i, column=1).font = Font(size=12, bold=True)

for i in range(1, ws.max_row + 1):
    for j in range(1, ws.max_column + 1):
        ws.cell(row=1, column=j).font = Font(size=12, bold=True)

for i in range(1, ws.max_row + 1):
    for j in range(1, ws.max_column + 1):
        ws.cell(row=i, column=j).border = Border(left=Side(border_style='thin', color='000000'),
                                                right=Side(border_style='thin', color='000000'),
                                                top=Side(border_style='thin', color='000000'),
                                                bottom=Side(border_style='thin', color='000000'))

wb.save(f'{os.getcwd()}/modified_export_2022.xlsx')

In [1298]:
exports_combined_df

,Pipe TTNr,Total
1,87167644199,14490.0
2,87167638629,13400.0
3,8718672348T03,12800.0
4,8750724192T03,11250.0
5,8718674696T03,7750.0
...,...,...
63,87107158299,100.0
64,87107251375,100.0
65,87107250849,100.0
66,8705900537T03,50.0
